# Imports

# This is a modification of the evaluate_target_network_on_evaluation file

### run this only if you dont have these folders

In [4]:
# %%bash
# mkdir results
# mkdir formatted_datasets

In [20]:
import numpy as np
import copy
import time

import torch
from torch.utils.data import TensorDataset
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable

import target_network_raw_emg_enhanced
import load_pre_training_dataset
import load_evaluation_dataset

## Method to shuffle examples and labels

As we went over before, shuffling is used to introduce randomness and reduce the chances that the model will be able to learn any bias in the order of the training data set.

In [21]:
def scramble(examples, labels, second_labels=[]):
    random_vec = np.arange(len(labels))
    np.random.shuffle(random_vec)
    new_labels = []
    new_examples = []
    if len(second_labels) == len(labels):
        new_second_labels = []
        for i in random_vec:
            new_labels.append(labels[i])
            new_examples.append(examples[i])
            new_second_labels.append(second_labels[i])
        return new_examples, new_labels, new_second_labels
    else:
        for i in random_vec:
            new_labels.append(labels[i])
            new_examples.append(examples[i])
        return new_examples, new_labels

### Method to calculate the pre training network

The reason begind the j in range 19 is that there are 19 patients
The reason that we have the k<21 is that there should be 5 times 7=35 total entries for data points and we want to use the first 3 times 7=21 for training and the rest for validation

In [22]:
def calculate_pre_training(examples, labels):
    list_train_dataloader = []
    list_validation_dataloader = []
    human_number = 0
    for j in range(19):
        # 19 patients 
        examples_personne_training = []
        labels_gesture_personne_training = []
        labels_human_personne_training = []

        examples_personne_valid = []
        labels_gesture_personne_valid = []
        labels_human_personne_valid = []

        for k in range(len(examples[j])):
            if k < 21:
                examples_personne_training.extend(examples[j][k])
                labels_gesture_personne_training.extend(labels[j][k])
                labels_human_personne_training.extend(human_number * np.ones(len(labels[j][k])))
            else:
                examples_personne_valid.extend(examples[j][k])
                labels_gesture_personne_valid.extend(labels[j][k])
                labels_human_personne_valid.extend(human_number * np.ones(len(labels[j][k])))

        print(np.shape(examples_personne_training))
        examples_personne_scrambled, labels_gesture_personne_scrambled, labels_human_personne_scrambled = scramble(
            examples_personne_training, labels_gesture_personne_training, labels_human_personne_training)

        examples_personne_scrambled_valid, labels_gesture_personne_scrambled_valid, labels_human_personne_scrambled_valid = scramble(
            examples_personne_valid, labels_gesture_personne_valid, labels_human_personne_valid)

        
        # Create the training and validaiton datasets
        train = TensorDataset(torch.from_numpy(np.array(examples_personne_scrambled, dtype=np.float32)),
                              torch.from_numpy(np.array(labels_gesture_personne_scrambled, dtype=np.int64)))
        validation = TensorDataset(torch.from_numpy(np.array(examples_personne_scrambled_valid, dtype=np.float32)),
                                   torch.from_numpy(np.array(labels_gesture_personne_scrambled_valid, dtype=np.int64)))

        trainLoader = torch.utils.data.DataLoader(train, batch_size=3315, shuffle=True, drop_last=True)
        validationLoader = torch.utils.data.DataLoader(validation, batch_size=1312, shuffle=True, drop_last=True)

        list_train_dataloader.append(trainLoader)
        list_validation_dataloader.append(validationLoader)

        human_number += 1
        print("Shape training : ", np.shape(examples_personne_scrambled))
        print("Shape valid : ", np.shape(examples_personne_scrambled_valid))

    cnn = target_network_raw_emg_enhanced.SourceNetwork(number_of_class=7, dropout_rate=.35).cuda()

    criterion = nn.CrossEntropyLoss(size_average=False)
    optimizer = optim.Adam(cnn.parameters(), lr=0.002335721469090121)
    precision = 1e-8
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer=optimizer, mode='min', factor=.2, patience=15,
                                                     verbose=True, eps=precision)

    pre_train_model(cnn, criterion=criterion, optimizer=optimizer, scheduler=scheduler,
                    dataloaders={"train": list_train_dataloader, "val": list_validation_dataloader},
                    precision=precision)

# Pre train model
This is a method that is used to pre train the source network.  At the end this prints out the best validation loss and then saves the cnn weights to a file so that they can be accessed later. 

In [23]:
def pre_train_model(cnn, criterion, optimizer, scheduler, dataloaders, num_epochs=500, precision=1e-8):
    since = time.time()

    # Create a list of dictionaries that will hold the weights of the batch normalisation layers for each dataset
    #  (i.e. each participants)
    list_dictionaries_BN_weights = []
    for index_BN_weights in range(len(dataloaders['val'])):
        state_dict = cnn.state_dict()
        batch_norm_dict = {}
        for key in state_dict:
            if "batch_norm" in key:
                batch_norm_dict.update({key: state_dict[key]})
        list_dictionaries_BN_weights.append(copy.deepcopy(batch_norm_dict))

    best_loss = float('inf')

    best_weights = copy.deepcopy(cnn.state_dict())

    patience = 30
    patience_increase = 30
    for epoch in range(num_epochs):
        epoch_start = time.time()
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                cnn.train(True)  # Set model to training mode
            else:
                cnn.train(False)  # Set model to evaluate mode

            running_loss = 0.
            running_corrects = 0
            total = 0

            # Get a random order for the training dataset
            random_vec = np.arange(len(dataloaders[phase]))
            np.random.shuffle(random_vec)

            for dataset_index in random_vec:
                # Retrieves the BN weights calculated so far for this dataset
                BN_weights = list_dictionaries_BN_weights[dataset_index]
                cnn.load_state_dict(BN_weights, strict=False)

                loss_over_datasets = 0.
                correct_over_datasets = 0.
                for i, data in enumerate(dataloaders[phase][dataset_index], 0):
                    # get the inputs
                    inputs, labels = data

                    inputs, labels = Variable(inputs.cuda()), Variable(labels.cuda())

                    # zero the parameter gradients
                    optimizer.zero_grad()
                    if phase == 'train':
                        cnn.train()
                        # forward
                        outputs = cnn(inputs)
                        _, predictions = torch.max(outputs.data, 1)
                        # backward
                        loss = criterion(outputs, labels)
                        loss.backward()
                        optimizer.step()
                        loss = loss.item()

                    else:
                        cnn.eval()
                        # forward
                        outputs = cnn(inputs)
                        _, predictions = torch.max(outputs.data, 1)
                        loss = criterion(outputs, labels)
                        loss = loss.item()
                    # Statistic for this dataset
                    loss_over_datasets += loss
                    correct_over_datasets += torch.sum(predictions == labels.data)
                    total += labels.size(0)
                # Statistic global
                running_loss += loss_over_datasets
                running_corrects += correct_over_datasets

                # Save the BN statistics for this dataset
                state_dict = cnn.state_dict()
                batch_norm_dict = {}
                for key in state_dict:
                    if "batch_norm" in key:
                        batch_norm_dict.update({key: state_dict[key]})
                list_dictionaries_BN_weights[dataset_index] = copy.deepcopy(batch_norm_dict)

            epoch_loss = running_loss / total
            epoch_acc = running_corrects.item() / total
            print('{} Loss: {:.8f} Acc: {:.8f}'.format(
                phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'val':
                scheduler.step(epoch_loss)
                if epoch_loss + precision < best_loss:
                    print("New best validation loss:", epoch_loss)
                    best_loss = epoch_loss
                    best_weights = copy.deepcopy(cnn.state_dict())
                    patience = patience_increase + epoch
            print("Epoch {} of {} took {:.3f}s".format(
                epoch + 1, num_epochs, time.time() - epoch_start))
        if epoch > patience:
            break

    print()

    time_elapsed = time.time() - since

    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val loss: {:4f}'.format(best_loss))

    # Save the best weights found to file
    torch.save(best_weights, 'convnet_weights/best_pre_train_weights_target_raw.pt')

# Calculate Fitness
This is a very large method that is involved in training the target network for all 17 of the individuals. The examples and labels are self explanatory, but it is important to see that for some reason there are two testing sets. The parameter for the training cycle is an integer that is used to see how many of each movement should be included in the training set. making this number equal to 4 will ensure that all the data is used. Changing this variable would be a good way to test how much training data the network "needs" from each new individual to be accurate enough.

At the end, it prints the average on test set 0 and 1 averaged over all 17 of the individuals.


In [24]:
def calculate_fitness(examples_training, labels_training, examples_test0, labels_test0, examples_test1, labels_test_1,
                      learning_rate=.1, training_cycle=4):
    accuracy_test0 = []
    accuracy_test1 = []
    for j in range(17): #17 individuals
        print("CURRENT DATASET : ", j)
        examples_personne_training = []
        labels_gesture_personne_training = []
        
        for k in range(len(examples_training[j])):
            if k < training_cycle * 7:
                examples_personne_training.extend(examples_training[j][k])
                labels_gesture_personne_training.extend(labels_training[j][k])
        
        X_test_0, Y_test_0 = [], []
        for k in range(len(examples_test0)):
            X_test_0.extend(examples_test0[j][k])
            Y_test_0.extend(labels_test0[j][k])
        
        X_test_1, Y_test_1 = [], []
        for k in range(len(examples_test1)):
            X_test_1.extend(examples_test1[j][k])
            Y_test_1.extend(labels_test_1[j][k])
            
        if training_cycle == 0:
            cnn = target_network_raw_emg_enhanced.SourceNetwork(number_of_class=7, dropout_rate=.35).cuda()
            cnn.eval()
            X_test_0, Y_test_0 = scramble(X_test_0, Y_test_0)

            test_0 = TensorDataset(torch.from_numpy(np.array(X_test_0, dtype=np.float32)),
                                   torch.from_numpy(np.array(Y_test_0, dtype=np.int64)))

            X_test_1, Y_test_1 = scramble(X_test_1, Y_test_1)

            test_1 = TensorDataset(torch.from_numpy(np.array(X_test_1, dtype=np.float32)),
                                   torch.from_numpy(np.array(Y_test_1, dtype=np.int64)))

            test_0_loader = torch.utils.data.DataLoader(test_0, batch_size=256, shuffle=False)
            total = 0
            correct_prediction_test_0 = 0
            for k, data_test_0 in enumerate(test_0_loader, 0):
                # get the inputs
                inputs_test_0, ground_truth_test_0 = data_test_0
                inputs_test_0, ground_truth_test_0 = Variable(inputs_test_0.cuda()), Variable(
                    ground_truth_test_0.cuda())
    
                outputs_test_0 = cnn(inputs_test_0)
                _, predicted = torch.max(outputs_test_0.data, 1)
                correct_prediction_test_0 += (predicted.cpu().numpy() == ground_truth_test_0.data.cpu().numpy()).sum()
                total += ground_truth_test_0.size(0)
            print("ACCURACY TEST_0 FINAL : %.3f %%" % (100 * float(correct_prediction_test_0) / float(total)))
            accuracy_test0.append(100 * float(correct_prediction_test_0) / float(total))

            test_1_loader = torch.utils.data.DataLoader(test_1, batch_size=256, shuffle=False)
            total = 0
            correct_prediction_test_1 = 0
            for k, data_test_1 in enumerate(test_1_loader, 0):
                # get the inputs
                inputs_test_1, ground_truth_test_1 = data_test_1
                inputs_test_1, ground_truth_test_1 = Variable(inputs_test_1.cuda()), Variable(
                    ground_truth_test_1.cuda())
    
                outputs_test_1 = cnn(inputs_test_1)
                _, predicted = torch.max(outputs_test_1.data, 1)
                correct_prediction_test_1 += (predicted.cpu().numpy() == ground_truth_test_1.data.cpu().numpy()).sum()
                total += ground_truth_test_1.size(0)
            print("ACCURACY TEST_1 FINAL : %.3f %%" % (100 * float(correct_prediction_test_1) / float(total)))
            accuracy_test1.append(100 * float(correct_prediction_test_1) / float(total))
        else:
            print(np.shape(examples_personne_training))
            examples_personne_scrambled, labels_gesture_personne_scrambled = scramble(examples_personne_training,
                                                                                      labels_gesture_personne_training)
            valid_examples = examples_personne_scrambled[0:int(len(examples_personne_scrambled) * 0.1)]
            labels_valid = labels_gesture_personne_scrambled[0:int(len(labels_gesture_personne_scrambled) * 0.1)]
            
            X_fine_tune = examples_personne_scrambled[int(len(examples_personne_scrambled) * 0.1):]
            Y_fine_tune = labels_gesture_personne_scrambled[int(len(labels_gesture_personne_scrambled) * 0.1):]
            
            
            train = TensorDataset(torch.from_numpy(np.array(X_fine_tune, dtype=np.float32)),
                                  torch.from_numpy(np.array(Y_fine_tune, dtype=np.int64)))
            
            validation = TensorDataset(torch.from_numpy(np.array(valid_examples, dtype=np.float32)),
                                       torch.from_numpy(np.array(labels_valid, dtype=np.int64)))
            
            trainloader = torch.utils.data.DataLoader(train, batch_size=256, shuffle=True, drop_last=True)
            validationloader = torch.utils.data.DataLoader(validation, batch_size=128, shuffle=True, drop_last=True)
    
            pre_trained_weights = torch.load('convnet_weights/best_pre_train_weights_target_raw.pt')
            cnn = target_network_raw_emg_enhanced.TargetNetwork(number_of_class=7,
                                                                weights_pre_trained_convnet=pre_trained_weights,
                                                                dropout=.5).cuda()
            
            criterion = nn.CrossEntropyLoss(size_average=False)
            optimizer = optim.Adam(cnn.parameters(), lr=learning_rate)
            
            precision = 1e-6
            scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer=optimizer, mode='min', factor=.2, patience=5,
                                                             verbose=True, eps=precision)
            
            cnn = train_model(cnn, criterion, optimizer, scheduler, dataloaders={"train": trainloader,
                                                                                 "val": validationloader},
                              precision=precision)
            
            cnn.eval()
            X_test_0, Y_test_0 = scramble(X_test_0, Y_test_0)
            
            test_0 = TensorDataset(torch.from_numpy(np.array(X_test_0, dtype=np.float32)),
                                   torch.from_numpy(np.array(Y_test_0, dtype=np.int64)))
            
            X_test_1, Y_test_1 = scramble(X_test_1, Y_test_1)
            
            test_1 = TensorDataset(torch.from_numpy(np.array(X_test_1, dtype=np.float32)),
                                   torch.from_numpy(np.array(Y_test_1, dtype=np.int64)))
            
            test_0_loader = torch.utils.data.DataLoader(test_0, batch_size=256, shuffle=False)
            total = 0
            correct_prediction_test_0 = 0
            for k, data_test_0 in enumerate(test_0_loader, 0):
                # get the inputs
                inputs_test_0, ground_truth_test_0 = data_test_0
                inputs_test_0, ground_truth_test_0 = Variable(inputs_test_0.cuda()), Variable(ground_truth_test_0.cuda())
                
                outputs_test_0 = cnn(inputs_test_0)
                _, predicted = torch.max(outputs_test_0.data, 1)
                correct_prediction_test_0 += (predicted.cpu().numpy() == ground_truth_test_0.data.cpu().numpy()).sum()
                total += ground_truth_test_0.size(0)
            print("ACCURACY TEST_0 FINAL : %.3f %%" % (100 * float(correct_prediction_test_0) / float(total)))
            accuracy_test0.append(100 * float(correct_prediction_test_0) / float(total))
            
            test_1_loader = torch.utils.data.DataLoader(test_1, batch_size=256, shuffle=False)
            total = 0
            correct_prediction_test_1 = 0
            for k, data_test_1 in enumerate(test_1_loader, 0):
                # get the inputs
                inputs_test_1, ground_truth_test_1 = data_test_1
                inputs_test_1, ground_truth_test_1 = Variable(inputs_test_1.cuda()), Variable(ground_truth_test_1.cuda())
                
                outputs_test_1 = cnn(inputs_test_1)
                _, predicted = torch.max(outputs_test_1.data, 1)
                correct_prediction_test_1 += (predicted.cpu().numpy() == ground_truth_test_1.data.cpu().numpy()).sum()
                total += ground_truth_test_1.size(0)
            print("ACCURACY TEST_1 FINAL : %.3f %%" % (100 * float(correct_prediction_test_1) / float(total)))
            accuracy_test1.append(100 * float(correct_prediction_test_1) / float(total))
    
    print("AVERAGE ACCURACY TEST 0 %.3f" % np.array(accuracy_test0).mean())
    print("AVERAGE ACCURACY TEST 1 %.3f" % np.array(accuracy_test1).mean())
    return accuracy_test0, accuracy_test1

# Train Model
This takes in the dataloaders and the cnn and simply just trains the cnn and returns it.

In [25]:

def train_model(cnn, criterion, optimizer, scheduler, dataloaders, num_epochs=500, precision=1e-8):
    since = time.time()
    
    best_loss = float('inf')
    
    patience = 30
    patience_increase = 10
    
    best_weights = copy.deepcopy(cnn.state_dict())
    
    for epoch in range(num_epochs):
        epoch_start = time.time()
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)
        
        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                cnn.train(True)  # Set model to training mode
            else:
                cnn.train(False)  # Set model to evaluate mode
            
            running_loss = 0.
            running_corrects = 0
            total = 0
            
            for i, data in enumerate(dataloaders[phase], 0):
                # get the inputs
                inputs, labels = data
                inputs, labels = Variable(inputs.cuda()), Variable(labels.cuda())
                
                # zero the parameter gradients
                optimizer.zero_grad()
                if phase == 'train':
                    cnn.train()
                    # forward
                    outputs = cnn(inputs)
                    _, predictions = torch.max(outputs.data, 1)
                    # backward
                    loss = criterion(outputs, labels)
                    loss.backward()
                    optimizer.step()
                    loss = loss.item()
                
                else:
                    cnn.eval()
                    # forward
                    outputs = cnn(inputs)
                    _, predictions = torch.max(outputs.data, 1)
                    loss = criterion(outputs, labels)
                    loss = loss.item()
                
                # statistics
                running_loss += loss
                running_corrects += torch.sum(predictions == labels.data)
                total += labels.size(0)
        
            epoch_loss = running_loss / total
            epoch_acc = running_corrects.item() / total
            
            print('{} Loss: {} Acc: {}'.format(phase, epoch_loss, epoch_acc))
            
            # deep copy the model
            if phase == 'val':
                scheduler.step(epoch_loss)
                if epoch_loss + precision < best_loss:
                    print("New best validation loss:", epoch_loss)
                    best_loss = epoch_loss
                    best_weights = copy.deepcopy(cnn.state_dict())
                    patience = patience_increase + epoch
        print("Epoch {} of {} took {:.3f}s".format(
            epoch + 1, num_epochs, time.time() - epoch_start))
        if epoch > patience:
            break
    print()
    
    time_elapsed = time.time() - since
    
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val loss: {:4f}'.format(best_loss))
    # load best model weights
    cnn.load_state_dict(copy.deepcopy(best_weights))
    cnn.eval()
    return cnn

# Actual Training

### Load Pre-Training Dataset and Pre-Train

In [26]:
# Comment between here    
# examples, labels = load_pre_training_dataset.read_data('../../PreTrainingDataset')
# datasets = [examples, labels]

# np.save("formatted_datasets/saved_pre_training_dataset_spectrogram.npy", datasets)
# And here if the pre-training dataset was already processed and saved


In [15]:
datasets_pre_training = np.load("formatted_datasets/saved_pre_training_dataset_spectrogram.npy", encoding="bytes", allow_pickle=True)
examples_pre_training, labels_pre_training = datasets_pre_training
calculate_pre_training(examples_pre_training, labels_pre_training)

(3981, 1, 8, 52)
Shape training :  (3981, 1, 8, 52)
Shape valid :  (1330, 1, 8, 52)
(3983, 1, 8, 52)
Shape training :  (3983, 1, 8, 52)
Shape valid :  (1327, 1, 8, 52)
(3976, 1, 8, 52)
Shape training :  (3976, 1, 8, 52)
Shape valid :  (1328, 1, 8, 52)
(3981, 1, 8, 52)
Shape training :  (3981, 1, 8, 52)
Shape valid :  (1327, 1, 8, 52)
(3984, 1, 8, 52)
Shape training :  (3984, 1, 8, 52)
Shape valid :  (1328, 1, 8, 52)
(3983, 1, 8, 52)
Shape training :  (3983, 1, 8, 52)
Shape valid :  (1325, 1, 8, 52)
(3985, 1, 8, 52)
Shape training :  (3985, 1, 8, 52)
Shape valid :  (1329, 1, 8, 52)
(3986, 1, 8, 52)
Shape training :  (3986, 1, 8, 52)
Shape valid :  (1326, 1, 8, 52)
(3982, 1, 8, 52)
Shape training :  (3982, 1, 8, 52)
Shape valid :  (1329, 1, 8, 52)
(3983, 1, 8, 52)
Shape training :  (3983, 1, 8, 52)
Shape valid :  (1327, 1, 8, 52)
(3984, 1, 8, 52)
Shape training :  (3984, 1, 8, 52)
Shape valid :  (1330, 1, 8, 52)
(3991, 1, 8, 52)
Shape training :  (3991, 1, 8, 52)
Shape valid :  (1327, 1,

/opt/conda/lib/python3.7/site-packages/torch/nn/_reduction.py:43: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))


Epoch 0/499
----------
train Loss: 1.53279836 Acc: 0.50474275
Epoch 1 of 500 took 6.537s
val Loss: 29.05439212 Acc: 0.47649898
New best validation loss: 29.054392117471878
Epoch 1 of 500 took 6.862s
Epoch 1/499
----------
train Loss: 0.85717918 Acc: 0.70955254
Epoch 2 of 500 took 1.380s
val Loss: 10.74891231 Acc: 0.56241531
New best validation loss: 10.74891231166638
Epoch 2 of 500 took 1.694s
Epoch 2/499
----------
train Loss: 0.66548084 Acc: 0.77769398
Epoch 3 of 500 took 1.480s
val Loss: 8.07471293 Acc: 0.53950711
New best validation loss: 8.074712932352128
Epoch 3 of 500 took 1.803s
Epoch 3/499
----------
train Loss: 0.61574299 Acc: 0.79889392
Epoch 4 of 500 took 1.550s
val Loss: 2.55770594 Acc: 0.59353828
New best validation loss: 2.557705941400554
Epoch 4 of 500 took 1.869s
Epoch 4/499
----------
train Loss: 0.59969415 Acc: 0.80000000
Epoch 5 of 500 took 1.495s
val Loss: 2.44739028 Acc: 0.65984925
New best validation loss: 2.447390279634212
Epoch 5 of 500 took 1.819s
Epoch 5/499


### Load evaluation data 

In [16]:
# Comment between here
# examples, labels = load_evaluation_dataset.read_data('../../EvaluationDataset', type="training0")
# datasets = [examples, labels]

# np.save("formatted_datasets/saved_evaluation_dataset_training.npy", datasets)

# examples, labels = load_evaluation_dataset.read_data('../../EvaluationDataset', type="Test0")
# datasets = [examples, labels]

# np.save("formatted_datasets/saved_evaluation_dataset_test0.npy", datasets)

# examples, labels = load_evaluation_dataset.read_data('../../EvaluationDataset', type="Test1")
# datasets = [examples, labels]

# np.save("formatted_datasets/saved_evaluation_dataset_test1.npy", datasets)

# And here if the pre-training dataset was already processed and saved

In [17]:
datasets_training = np.load("formatted_datasets/saved_evaluation_dataset_training.npy", encoding="bytes", allow_pickle=True)
examples_training, labels_training = datasets_training

datasets_test0 = np.load("formatted_datasets/saved_evaluation_dataset_test0.npy", encoding="bytes", allow_pickle=True)
examples_test0, labels_test0 = datasets_test0

datasets_test1 = np.load("formatted_datasets/saved_evaluation_dataset_test1.npy", encoding="bytes", allow_pickle=True)
examples_test1, labels_test1 = datasets_test1

### This next part actually returns the results
This part simply runs calculate fitness a given number of times and prints the results of the trials to the screen. The actual results that are meaningful are printed out to a text file for better reading.

In [18]:
accuracy_one_by_one = []
array_training_error = []
array_validation_error = []
# learning_rate=0.002335721469090121 (for network enhanced)


# this is the number of times that the tests are repeated in full, more repeats means less chance for random variance
number_repeated_tests=2

with open("results/evaluation_dataset_TARGET_convnet_enhanced.txt", "a") as myfile:
    myfile.write("Test")
    
    
# I changed it so it only runs the trial with number 4

    
for training_cycle in range(4, 5):
    test_0 = []
    test_1 = []
    for i in range(number_repeated_tests):
        accuracy_test0, accuracy_test1 = calculate_fitness(examples_training, labels_training, examples_test0,
                                                           labels_test0, examples_test1, labels_test1,
                                                           learning_rate=0.002335721469090121,
                                                           training_cycle=training_cycle)

        test_0.append(accuracy_test0)
        test_1.append(accuracy_test1)
        print("TEST 0 SO FAR: ", test_0)
        print("TEST 1 SO FAR: ", test_1)
        print("CURRENT AVERAGE : ", (np.mean(test_0) + np.mean(test_1)) / 2.)

    print("ACCURACY FINAL TEST 0: ", test_0)
    print("ACCURACY FINAL TEST 0: ", np.mean(test_0))
    print("ACCURACY FINAL TEST 1: ", test_1)
    print("ACCURACY FINAL TEST 1: ", np.mean(test_1))
    print("ACCURACY FINAL: ", (np.mean(test_0) + np.mean(test_1)) / 2.)

    with open("results/evaluation_dataset_TARGET_convnet_enhanced.txt", "a") as myfile:
        myfile.write("ConvNet Training Cycle : " + str(training_cycle) + "\n\n")
        myfile.write("Test 0: \n")
        myfile.write(str(test_0) + '\n')
        myfile.write(str(np.mean(test_0, axis=0)) + '\n')
        myfile.write(str(np.mean(test_0)) + '\n')
        myfile.write("Test 1: \n")
        myfile.write(str(test_1) + '\n')
        myfile.write(str(np.mean(test_1, axis=0)) + '\n')
        myfile.write(str(np.mean(test_1)) + '\n')
        myfile.write("Test Mean: \n")
        myfile.write(str(np.mean(test_0, axis=0)) + '\n')
        myfile.write(str((np.mean(test_0) + np.mean(test_1)) / 2.) + '\n')
        myfile.write("\n\n\n")


CURRENT DATASET :  0
(5309, 1, 8, 52)
SourceNetwork(
  (_conv1): Conv2d(1, 32, kernel_size=(3, 5), stride=(1, 1))
  (_pool1): MaxPool2d(kernel_size=(1, 3), stride=(1, 3), padding=0, dilation=1, ceil_mode=False)
  (_batch_norm1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (_prelu1): PReLU(num_parameters=32)
  (_dropout1): Dropout2d(p=0.5, inplace=False)
  (_conv2): Conv2d(32, 64, kernel_size=(3, 5), stride=(1, 1))
  (_pool2): MaxPool2d(kernel_size=(1, 3), stride=(1, 3), padding=0, dilation=1, ceil_mode=False)
  (_batch_norm2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (_prelu2): PReLU(num_parameters=64)
  (_dropout2): Dropout2d(p=0.5, inplace=False)
  (_fc1): Linear(in_features=1024, out_features=500, bias=True)
  (_batch_norm3): BatchNorm1d(500, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (_prelu3): PReLU(num_parameters=500)
  (_dropout3): Dropout(p=0.5, inplace=False)
  (_output): Line

/home/jovyan/work/eNABLE_repo/myoArmBandDataset/MyoArmbandDataset/PyTorchImplementation/RawEnhancedConvNet/target_network_raw_emg_enhanced.py:123: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.
  torch.nn.init.kaiming_normal(m.weight)
/home/jovyan/work/eNABLE_repo/myoArmBandDataset/MyoArmbandDataset/PyTorchImplementation/RawEnhancedConvNet/target_network_raw_emg_enhanced.py:126: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.
  torch.nn.init.kaiming_normal(m.weight)


train Loss: 0.5149335964686341 Acc: 0.8361545138888888
val Loss: 0.040895371697843075 Acc: 0.986328125
New best validation loss: 0.040895371697843075
Epoch 1 of 500 took 0.570s
Epoch 1/499
----------
train Loss: 0.08209535396761364 Acc: 0.97265625
val Loss: 0.015548140276223421 Acc: 0.99609375
New best validation loss: 0.015548140276223421
Epoch 2 of 500 took 0.401s
Epoch 2/499
----------
train Loss: 0.04208566351897187 Acc: 0.9835069444444444
val Loss: 0.004912353120744228 Acc: 1.0
New best validation loss: 0.004912353120744228
Epoch 3 of 500 took 0.377s
Epoch 3/499
----------
train Loss: 0.03519743251510792 Acc: 0.9887152777777778
val Loss: 0.0036725192330777645 Acc: 1.0
New best validation loss: 0.0036725192330777645
Epoch 4 of 500 took 0.364s
Epoch 4/499
----------
train Loss: 0.0309506653704577 Acc: 0.98828125
val Loss: 0.0023973220959305763 Acc: 1.0
New best validation loss: 0.0023973220959305763
Epoch 5 of 500 took 0.342s
Epoch 5/499
----------
train Loss: 0.025588678144332435 A

# End of File

**Check `results/evaluation_dataset_TARGET_convnet_enhanced.txt` for helpful metrics. Most of it will appear as arrays that are 17 long, indicating the 17 patients that the results are averaged over.**